## Package Install

In [354]:
# Programming
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Modelling
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm
from fbprophet import Prophet
from sklearn.model_selection import ParameterGrid
import xgboost as xgb


# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')


## Data Preprocessing

In [368]:
# Import Data
feature = pd.read_csv('features.csv', index_col = 'Date', parse_dates = True)\
            .drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis = 1)
store = pd.read_csv('stores.csv')
store.Type = [0 if i == 'A' else 1 for i in store.Type]
trainall = pd.read_csv('train_all.csv').drop(['Unnamed: 0','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis = 1)


# merge training and feature dataset
trainall = trainall.groupby(['Store','Date']).agg({'Weekly_Sales':'sum'}).reset_index()
trainall.Date = pd.to_datetime(trainall.Date)
training = trainall.merge(feature, 'inner', on = ['Store', 'Date']).merge(store, 'inner', on = 'Store').rename(columns = {'IsHoliday_x':'IsHoliday'}).set_index('Date')


#renaming datetime/target variable
training_pro = training[training['Store'] == 1].iloc[:, 1:].drop(['IsHoliday','Type','Size'], axis = 1).reset_index().rename(columns = {'Date' : 'ds', 'Weekly_Sales' : 'y'})




## Modeling - Prophet

In [930]:
# Parameter grid
param_grid = {'seasonality_prior_scale': [5, 10, 20],
              'changepoint_prior_scale': [0.01, 0.05, 0.1],
              'holidays_prior_scale': [5, 10, 20]}
grid = ParameterGrid(param_grid)

#Hyper parameter tuning

rmse_container = []
mape_container = []

for params in grid:
    rmse = []
    mape = []
    for i in range(45):
        
        # Renaming variable
        training_pro = training[training['Store'] == i+1].iloc[:, 1:].drop(['IsHoliday','Type','Size'], axis = 1).reset_index().rename(columns = {'Date' : 'ds', 'Weekly_Sales' : 'y'})
        testing_pro = testing[testing['Store'] == i+1].iloc[:, 1:].drop(['IsHoliday','Type','Size'], axis = 1).reset_index().rename(columns = {'Date' : 'ds'})

        # Splitting training and validation set
        test_days = 30
        training_set = training_pro.iloc[:-test_days, :]
        validation_set = training_pro.iloc[-test_days:, :].drop('y', axis = 1)

        # Creating holiday frame
        holiday_dates = training[(training['Store'] == i+1)&(training.IsHoliday == True)].reset_index().rename(columns = {'Date' : 'ds', 'Weekly_Sales' : 'y'}).ds
        holidays = pd.DataFrame({'holiday' : 'holi',
                                 'ds': pd.to_datetime(holiday_dates),
                                 'lower_window': -3,
                                 'upper_window': 1})
        
        # Building model
        m = Prophet(growth = "linear",
                yearly_seasonality = True,
                weekly_seasonality = True,
                daily_seasonality = False,
                holidays = holidays,
                seasonality_mode = "multiplicative",
                seasonality_prior_scale = params['seasonality_prior_scale'],
                holidays_prior_scale = params['holidays_prior_scale'],
                changepoint_prior_scale = params['changepoint_prior_scale'])
        m.add_regressor('Temperature')
        m.add_regressor('Fuel_Price')
        m.add_regressor('CPI')
        m.add_regressor('Unemployment')
        m.fit(training_set)

        # Cross-validation
        df_cv = cross_validation(m,
                             period='31 days', 
                             horizon = '150 days',
                             initial = '600 days',
                             parallel = "processes")

        # Gathering the rmse & MAPE
        error = np.sqrt(mean_squared_error(df_cv['y'], df_cv['yhat']))
        mape_error = MAPE(df_cv['y'], df_cv['yhat'])
        
        rmse.append(error)
        mape.append(mape_error)
    rmse_container.append(rmse)
    mape_container.append(mape)
    

# Show the minimum average MAPE for 45 Stores
avg_mape_for_all_param = [sum(mape)/45 for mape in mape_container]
print(avg_mape_for_all_param.min())

        

INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83289dc10>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83579ed30>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8c55df1f0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe9cbcdf8e0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <con

INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe82f70ec40>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8328656a0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe9e010e3a0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe82f6d5850>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8328a27c0>
INFO:fbprophet:Making 2 forec

INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe9d23f7790>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f6a24f0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe87c12b4c0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83289d6d0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <con

INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe832896670>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe845f8e1c0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe9e0100a00>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe97d55e130>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe832891c70>
INFO:fbprophet:Making 2 forec

INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe84c72a700>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f6afb80>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2ed52550>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe832899a90>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <con

...

INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83289dd30>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83288a4c0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69ba00>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69bf70>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe832896a30>
INFO:fbprophet:Making 2 forec

INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe97d564880>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe9bdbc6ee0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b820>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b700>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <con

INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8328708e0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b310>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b160>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83577bc10>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f6af790>
INFO:fbprophet:Making 2 forec

INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b040>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8c55e9760>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe8c55e9760>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe832899af0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <con

INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe92ed6d760>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69bbb0>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe83288a940>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fea2f69b160>
INFO:fbprophet:Making 2 forecasts with cutoffs between 2011-10-01 00:00:00 and 2011-11-01 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fe84807aa60>
INFO:fbprophet:Making 2 forec

## Ensemble Modeling: Prophet+XGBoost

In [1012]:
#Set the parameters
parameters = {'learning_rate': 0.1,
              'max_depth': 3,
              'colsample_bytree': 1,
              'subsample': 1,
              'min_child_weight': 1,
              'gamma': 1,
              'random_state': 1502,
              'eval_metric': "rmse",
              'objective': "reg:squarederror"}

# Fitting best parameter for the model
m = Prophet(growth = "linear",
                yearly_seasonality = True,
                weekly_seasonality = True,
                daily_seasonality = False,
                holidays = holidays,
                seasonality_mode = "multiplicative",
                seasonality_prior_scale = 10,
                holidays_prior_scale = 20,
                changepoint_prior_scale = 0.1)
m.add_regressor('Temperature')
m.add_regressor('Fuel_Price')
m.add_regressor('CPI')
m.add_regressor('Unemployment')
m.fit(training_set)

# Building 45 models for 45 stores
rmse = []
mape = []

for i in range(45):

    training_pro = training[training['Store'] == i+1].iloc[:, 1:].drop(['IsHoliday','Type','Size'], axis = 1).reset_index().rename(columns = {'Date' : 'ds', 'Weekly_Sales' : 'y'})
    testing_pro = testing[testing['Store'] == i+1].iloc[:, 1:].drop(['IsHoliday','Type','Size'], axis = 1).reset_index().rename(columns = {'Date' : 'ds'})

    # Splitting training and validation set for Prophet prediction
    test_days = 30
    training_set = training_pro.iloc[:-test_days, :]
    validation_set = training_pro.iloc[-test_days:, :].drop('y', axis = 1)

    forecast_xgb = m.predict(training_pro.drop('y', axis = 1))

    # Combining the original dataset with the prophet variable
    prophet_variables = forecast_xgb.loc[:, ["trend", "holi", "weekly", "yearly"]]
    df_xgb = pd.concat([training_pro, prophet_variables], axis = 1)

    # Splitting training and test set for XGBoost prediction
    test_days = 30
    training_set = df_xgb.iloc[:-test_days, :]
    validation_set = df_xgb.iloc[-test_days:, :]

    # Isolating X and Y
    y_train = training_set.y
    y_val = validation_set.y
    X_train = training_set.iloc[:, 2:]
    X_val = validation_set.iloc[:, 2:]

    # Creating XGBoost Matrices
    Train = xgb.DMatrix(data = X_train, label = y_train)
    Test = xgb.DMatrix(data = X_val, label = y_val)

    # Building XGBoost Model
    model = xgb.train(params = parameters,
                      dtrain = Train,
                      num_boost_round = 200, #166
                      evals = [(Test, "y")],
                      verbose_eval = 15)

    # Predicting with XGBoost
    predictions_xgb = pd.Series(model.predict(Test), name = "XGBoost")
    predictions_xgb.index = validation_set.ds
    predictions_xgb
    
    # Setting the index
    training_set.index = training_set.ds
    validation_set.index = validation_set.ds

    # Gathering the rmse & MAPE
    error = np.sqrt(mean_squared_error(validation_set['y'], predictions_xgb))
    mape_error = MAPE(validation_set['y'], predictions_xgb)
    rmse.append(error)
    mape.append(mape_error)

# Show the average MAPE for 45 Stores
print(sum(mape)/45)


[0]	y-rmse:1.43252e+06
[15]	y-rmse:378436
[30]	y-rmse:170255
[45]	y-rmse:138600
[60]	y-rmse:134463
[75]	y-rmse:134375
[90]	y-rmse:134962
[105]	y-rmse:134832
[120]	y-rmse:135428
[135]	y-rmse:136606
[150]	y-rmse:138072
[165]	y-rmse:138894
[180]	y-rmse:139325
[195]	y-rmse:140521
[199]	y-rmse:140400
[0]	y-rmse:1.70944e+06
[15]	y-rmse:401022
[30]	y-rmse:164004
[45]	y-rmse:108956
[60]	y-rmse:101762
[75]	y-rmse:100980
[90]	y-rmse:98826.2
[105]	y-rmse:99925.4
[120]	y-rmse:102812
[135]	y-rmse:103273
[150]	y-rmse:104422
[165]	y-rmse:105364
[180]	y-rmse:107190
[195]	y-rmse:107975
[199]	y-rmse:108159
[0]	y-rmse:381266
[15]	y-rmse:111344
[30]	y-rmse:46256.4
[45]	y-rmse:35323.6
[60]	y-rmse:32568.3
[75]	y-rmse:31902.2
[90]	y-rmse:30818.8
[105]	y-rmse:29568.1
[120]	y-rmse:29393.4
[135]	y-rmse:28898.3
[150]	y-rmse:29146.4
[165]	y-rmse:28749.6
[180]	y-rmse:28602.2
[195]	y-rmse:28594.5
[199]	y-rmse:28508.5
[0]	y-rmse:1.95721e+06
[15]	y-rmse:485286
[30]	y-rmse:156822
[45]	y-rmse:93471.2
[60]	y-rmse:88014.

[60]	y-rmse:131488
[75]	y-rmse:139204
[90]	y-rmse:139690
[105]	y-rmse:140787
[120]	y-rmse:142782
[135]	y-rmse:143288
[150]	y-rmse:142932
[165]	y-rmse:145660
[180]	y-rmse:148718
[195]	y-rmse:149371
[199]	y-rmse:149679
[0]	y-rmse:480274
[15]	y-rmse:120986
[30]	y-rmse:55272
[45]	y-rmse:42497.2
[60]	y-rmse:40079.8
[75]	y-rmse:39761.3
[90]	y-rmse:40783.6
[105]	y-rmse:40484.6
[120]	y-rmse:41626.1
[135]	y-rmse:41437.7
[150]	y-rmse:41001.2
[165]	y-rmse:41259.1
[180]	y-rmse:40763.7
[195]	y-rmse:40418.8
[199]	y-rmse:40196.4
[0]	y-rmse:393160
[15]	y-rmse:84985.8
[30]	y-rmse:28379.7
[45]	y-rmse:21065.6
[60]	y-rmse:20012.3
[75]	y-rmse:16653.4
[90]	y-rmse:17086
[105]	y-rmse:16151
[120]	y-rmse:15844.4
[135]	y-rmse:15198.9
[150]	y-rmse:15102.7
[165]	y-rmse:15676.7
[180]	y-rmse:16102.8
[195]	y-rmse:16245.4
[199]	y-rmse:16274.7
[0]	y-rmse:1.24741e+06
[15]	y-rmse:269903
[30]	y-rmse:88572.3
[45]	y-rmse:51854.8
[60]	y-rmse:43414
[75]	y-rmse:42221.6
[90]	y-rmse:41623.3
[105]	y-rmse:41752
[120]	y-rmse:42251.

[105]	y-rmse:109212
[120]	y-rmse:111514
[135]	y-rmse:114141
[150]	y-rmse:118551
[165]	y-rmse:120431
[180]	y-rmse:122633
[195]	y-rmse:123763
[199]	y-rmse:123790
[0]	y-rmse:1.21723e+06
[15]	y-rmse:294776
[30]	y-rmse:120201
[45]	y-rmse:91549.9
[60]	y-rmse:84700.6
[75]	y-rmse:84763.2
[90]	y-rmse:84443.9
[105]	y-rmse:84237.7
[120]	y-rmse:83639.8
[135]	y-rmse:83354.3
[150]	y-rmse:82790.9
[165]	y-rmse:83403.6
[180]	y-rmse:83465.8
[195]	y-rmse:83333.6
[199]	y-rmse:83386.4
[0]	y-rmse:893094
[15]	y-rmse:213199
[30]	y-rmse:59846.1
[45]	y-rmse:49196.4
[60]	y-rmse:49459.6
[75]	y-rmse:49984.5
[90]	y-rmse:51144.6
[105]	y-rmse:52036.1
[120]	y-rmse:51884
[135]	y-rmse:52057.7
[150]	y-rmse:52051.1
[165]	y-rmse:52204.2
[180]	y-rmse:52917.9
[195]	y-rmse:53465.2
[199]	y-rmse:53484.5
[0]	y-rmse:1.85534e+06
[15]	y-rmse:488919
[30]	y-rmse:187536
[45]	y-rmse:134138
[60]	y-rmse:126897
[75]	y-rmse:128852
[90]	y-rmse:130417
[105]	y-rmse:132923
[120]	y-rmse:132766
[135]	y-rmse:135974
[150]	y-rmse:139919
[165]	y-rms

[150]	y-rmse:22874.4
[165]	y-rmse:23012.3
[180]	y-rmse:23268.8
[195]	y-rmse:23228.7
[199]	y-rmse:23292.4
[0]	y-rmse:396810
[15]	y-rmse:106358
[30]	y-rmse:42881.9
[45]	y-rmse:34820.9
[60]	y-rmse:34137.3
[75]	y-rmse:33590
[90]	y-rmse:34587.8
[105]	y-rmse:34675.5
[120]	y-rmse:34544.3
[135]	y-rmse:34922.3
[150]	y-rmse:35165.5
[165]	y-rmse:35639.3
[180]	y-rmse:35758.6
[195]	y-rmse:35766.3
[199]	y-rmse:35761
[0]	y-rmse:1.43353e+06
[15]	y-rmse:433313
[30]	y-rmse:212270
[45]	y-rmse:172521
[60]	y-rmse:163680
[75]	y-rmse:164565
[90]	y-rmse:166696
[105]	y-rmse:167363
[120]	y-rmse:169917
[135]	y-rmse:170748
[150]	y-rmse:171412
[165]	y-rmse:172861
[180]	y-rmse:175004
[195]	y-rmse:175586
[199]	y-rmse:176011
[0]	y-rmse:891262
[15]	y-rmse:226515
[30]	y-rmse:114991
[45]	y-rmse:99216.1
[60]	y-rmse:96923.6
[75]	y-rmse:93123.5
[90]	y-rmse:90595.2
[105]	y-rmse:89122.5
[120]	y-rmse:88075.8
[135]	y-rmse:87536.7
[150]	y-rmse:89591.7
[165]	y-rmse:89244.4
[180]	y-rmse:88556.4
[195]	y-rmse:87600.8
[199]	y-rmse:8

[195]	y-rmse:90973.5
[199]	y-rmse:90863.9
[0]	y-rmse:1.89075e+06
[15]	y-rmse:476455
[30]	y-rmse:203558
[45]	y-rmse:159543
[60]	y-rmse:150867
[75]	y-rmse:151184
[90]	y-rmse:152805
[105]	y-rmse:153430
[120]	y-rmse:156115
[135]	y-rmse:156058
[150]	y-rmse:157219
[165]	y-rmse:157657
[180]	y-rmse:158518
[195]	y-rmse:158552
[199]	y-rmse:158589
[0]	y-rmse:626439
[15]	y-rmse:105598
[30]	y-rmse:41601.2
[45]	y-rmse:52159.1
[60]	y-rmse:54530.5
[75]	y-rmse:52761.1
[90]	y-rmse:52602.6
[105]	y-rmse:52742.7
[120]	y-rmse:52308.6
[135]	y-rmse:52352.7
[150]	y-rmse:52648.7
[165]	y-rmse:52472.4
[180]	y-rmse:52252.9
[195]	y-rmse:52287.2
[199]	y-rmse:52299.4
[0]	y-rmse:918258
[15]	y-rmse:222639
[30]	y-rmse:87306.5
[45]	y-rmse:69144.4
[60]	y-rmse:66310.3
[75]	y-rmse:65583.8
[90]	y-rmse:66457.8
[105]	y-rmse:66507.6
[120]	y-rmse:66291.2
[135]	y-rmse:67516.3
[150]	y-rmse:67845.8
[165]	y-rmse:67434
[180]	y-rmse:67149.1
[195]	y-rmse:66700.9
[199]	y-rmse:66645.5
[0]	y-rmse:1.29869e+06
[15]	y-rmse:363458
[30]	y-rmse

...

[199]	y-rmse:30687.6
[0]	y-rmse:1.61169e+06
[15]	y-rmse:331234
[30]	y-rmse:102647
[45]	y-rmse:100065
[60]	y-rmse:107059
[75]	y-rmse:105246
[90]	y-rmse:107921
[105]	y-rmse:109212
[120]	y-rmse:111514
[135]	y-rmse:114141
[150]	y-rmse:118551
[165]	y-rmse:120431
[180]	y-rmse:122633
[195]	y-rmse:123763
[199]	y-rmse:123790
[0]	y-rmse:1.21723e+06
[15]	y-rmse:294776
[30]	y-rmse:120201
[45]	y-rmse:91549.9
[60]	y-rmse:84700.6
[75]	y-rmse:84763.2
[90]	y-rmse:84443.9
[105]	y-rmse:84237.7
[120]	y-rmse:83639.8
[135]	y-rmse:83354.3
[150]	y-rmse:82790.9
[165]	y-rmse:83403.6
[180]	y-rmse:83465.8
[195]	y-rmse:83333.6
[199]	y-rmse:83386.4
[0]	y-rmse:893094
[15]	y-rmse:213199
[30]	y-rmse:59846.1
[45]	y-rmse:49196.4
[60]	y-rmse:49459.6
[75]	y-rmse:49984.5
[90]	y-rmse:51144.6
[105]	y-rmse:52036.1
[120]	y-rmse:51884
[135]	y-rmse:52057.7
[150]	y-rmse:52051.1
[165]	y-rmse:52204.2
[180]	y-rmse:52917.9
[195]	y-rmse:53465.2
[199]	y-rmse:53484.5
[0]	y-rmse:1.85534e+06
[15]	y-rmse:488919
[30]	y-rmse:187536
[45]	y-rm

[45]	y-rmse:22087.9
[60]	y-rmse:23116.8
[75]	y-rmse:22611.7
[90]	y-rmse:22152.3
[105]	y-rmse:22120.6
[120]	y-rmse:22059.6
[135]	y-rmse:22337.1
[150]	y-rmse:22874.4
[165]	y-rmse:23012.3
[180]	y-rmse:23268.8
[195]	y-rmse:23228.7
[199]	y-rmse:23292.4
[0]	y-rmse:396810
[15]	y-rmse:106358
[30]	y-rmse:42881.9
[45]	y-rmse:34820.9
[60]	y-rmse:34137.3
[75]	y-rmse:33590
[90]	y-rmse:34587.8
[105]	y-rmse:34675.5
[120]	y-rmse:34544.3
[135]	y-rmse:34922.3
[150]	y-rmse:35165.5
[165]	y-rmse:35639.3
[180]	y-rmse:35758.6
[195]	y-rmse:35766.3
[199]	y-rmse:35761
[0]	y-rmse:1.43353e+06
[15]	y-rmse:433313
[30]	y-rmse:212270
[45]	y-rmse:172521
[60]	y-rmse:163680
[75]	y-rmse:164565
[90]	y-rmse:166696
[105]	y-rmse:167363
[120]	y-rmse:169917
[135]	y-rmse:170748
[150]	y-rmse:171412
[165]	y-rmse:172861
[180]	y-rmse:175004
[195]	y-rmse:175586
[199]	y-rmse:176011
[0]	y-rmse:891262
[15]	y-rmse:226515
[30]	y-rmse:114991
[45]	y-rmse:99216.1
[60]	y-rmse:96923.6
[75]	y-rmse:93123.5
[90]	y-rmse:90595.2
[105]	y-rmse:89122

[105]	y-rmse:84826.1
[120]	y-rmse:86556.2
[135]	y-rmse:87089.6
[150]	y-rmse:88256.3
[165]	y-rmse:88907.2
[180]	y-rmse:90698.7
[195]	y-rmse:90973.5
[199]	y-rmse:90863.9
[0]	y-rmse:1.89075e+06
[15]	y-rmse:476455
[30]	y-rmse:203558
[45]	y-rmse:159543
[60]	y-rmse:150867
[75]	y-rmse:151184
[90]	y-rmse:152805
[105]	y-rmse:153430
[120]	y-rmse:156115
[135]	y-rmse:156058
[150]	y-rmse:157219
[165]	y-rmse:157657
[180]	y-rmse:158518
[195]	y-rmse:158552
[199]	y-rmse:158589
[0]	y-rmse:626439
[15]	y-rmse:105598
[30]	y-rmse:41601.2
[45]	y-rmse:52159.1
[60]	y-rmse:54530.5
[75]	y-rmse:52761.1
[90]	y-rmse:52602.6
[105]	y-rmse:52742.7
[120]	y-rmse:52308.6
[135]	y-rmse:52352.7
[150]	y-rmse:52648.7
[165]	y-rmse:52472.4
[180]	y-rmse:52252.9
[195]	y-rmse:52287.2
[199]	y-rmse:52299.4
[0]	y-rmse:918258
[15]	y-rmse:222639
[30]	y-rmse:87306.5
[45]	y-rmse:69144.4
[60]	y-rmse:66310.3
[75]	y-rmse:65583.8
[90]	y-rmse:66457.8
[105]	y-rmse:66507.6
[120]	y-rmse:66291.2
[135]	y-rmse:67516.3
[150]	y-rmse:67845.8
[165]	y-r

[150]	y-rmse:138072
[165]	y-rmse:138894
[180]	y-rmse:139325
[195]	y-rmse:140521
[199]	y-rmse:140400
[0]	y-rmse:1.70944e+06
[15]	y-rmse:401022
[30]	y-rmse:164004
[45]	y-rmse:108956
[60]	y-rmse:101762
[75]	y-rmse:100980
[90]	y-rmse:98826.2
[105]	y-rmse:99925.4
[120]	y-rmse:102812
[135]	y-rmse:103273
[150]	y-rmse:104422
[165]	y-rmse:105364
[180]	y-rmse:107190
[195]	y-rmse:107975
[199]	y-rmse:108159
[0]	y-rmse:381266
[15]	y-rmse:111344
[30]	y-rmse:46256.4
[45]	y-rmse:35323.6
[60]	y-rmse:32568.3
[75]	y-rmse:31902.2
[90]	y-rmse:30818.8
[105]	y-rmse:29568.1
[120]	y-rmse:29393.4
[135]	y-rmse:28898.3
[150]	y-rmse:29146.4
[165]	y-rmse:28749.6
[180]	y-rmse:28602.2
[195]	y-rmse:28594.5
[199]	y-rmse:28508.5
[0]	y-rmse:1.95721e+06
[15]	y-rmse:485286
[30]	y-rmse:156822
[45]	y-rmse:93471.2
[60]	y-rmse:88014.6
[75]	y-rmse:83823.1
[90]	y-rmse:83407.8
[105]	y-rmse:83070.8
[120]	y-rmse:84069.9
[135]	y-rmse:83323.4
[150]	y-rmse:83052
[165]	y-rmse:83531.2
[180]	y-rmse:83812.6
[195]	y-rmse:84234.5
[199]	y-rm

[199]	y-rmse:149679
[0]	y-rmse:480274
[15]	y-rmse:120986
[30]	y-rmse:55272
[45]	y-rmse:42497.2
[60]	y-rmse:40079.8
[75]	y-rmse:39761.3
[90]	y-rmse:40783.6
[105]	y-rmse:40484.6
[120]	y-rmse:41626.1
[135]	y-rmse:41437.7
[150]	y-rmse:41001.2
[165]	y-rmse:41259.1
[180]	y-rmse:40763.7
[195]	y-rmse:40418.8
[199]	y-rmse:40196.4
[0]	y-rmse:393160
[15]	y-rmse:84985.8
[30]	y-rmse:28379.7
[45]	y-rmse:21065.6
[60]	y-rmse:20012.3
[75]	y-rmse:16653.4
[90]	y-rmse:17086
[105]	y-rmse:16151
[120]	y-rmse:15844.4
[135]	y-rmse:15198.9
[150]	y-rmse:15102.7
[165]	y-rmse:15676.7
[180]	y-rmse:16102.8
[195]	y-rmse:16245.4
[199]	y-rmse:16274.7
[0]	y-rmse:1.24741e+06
[15]	y-rmse:269903
[30]	y-rmse:88572.3
[45]	y-rmse:51854.8
[60]	y-rmse:43414
[75]	y-rmse:42221.6
[90]	y-rmse:41623.3
[105]	y-rmse:41752
[120]	y-rmse:42251.1
[135]	y-rmse:41622.3
[150]	y-rmse:41629
[165]	y-rmse:41835.1
[180]	y-rmse:42427.7
[195]	y-rmse:41729.8
[199]	y-rmse:41401.7
[0]	y-rmse:1.07525e+06
[15]	y-rmse:269437
[30]	y-rmse:100074
[45]	y-rms